In [2]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from lightgbm import LGBMClassifier

import re
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import joblib


In [6]:
df = pd.read_json('/Users/kiyas/Documents/Programming_Practice/nlp-project/data-engineering/data/multi-label-news.json', orient='records', lines=True)

In [4]:
# Load the saved models
model = joblib.load('data/trained_model.joblib')


/opt/homebrew/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.6.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
txt = """
A federal judge, citing a risk of “irreparable harm,” has temporarily restricted Elon Musk’s government efficiency team from accessing a critical Treasury Department payment system.

The judge’s order, issued Saturday, temporarily halts access to a sensitive payment system that distributes Americans’ tax returns, Social Security benefits, disability payments and federal employees’ salaries.

US District Judge Paul Engelmayer ordered the destruction of any downloaded information from the payment system by anyone given access to it since January 20, citing “the risk that the new policy presents of the disclosure of sensitive and confidential information and the heightened risk that the systems in question will be more vulnerable than before to hacking.”

A hearing on the matter was set for February 14.

Engelmayer’s order came in response to a suit filed by New York Attorney General Letitia James and 18 other state attorneys general against the Trump administration.

The suit alleges the team led by Musk and staffed by young associates categorized as “special government employees” have been unlawfully granted access to the Treasury system that previously was restricted to specific government employees.

"""

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

def preprocess_text(text):
    '''Preprocess text by making it lowercase, removing text in square brackets,
removing links, removing punctuation, and removing words containing numbers.'''
    return re.sub('\[.*?\]|\w*\d\w*|https?://\S+|www\.\S+|<.*?>+|[%s]' %
re.escape(string.punctuation), '', str(text).lower())

def apply_stemming(sentence):
    stemmer = nltk.SnowballStemmer("english")
    return ' '.join(stemmer.stem(word) for word in sentence.split(' '))

def preprocess_and_clean(sentence):
    '''Preprocess and clean the text'''
    cleaned_text = preprocess_text(sentence)
    stop_words = stopwords.words('english')
    removed_stopwords_text = ' '.join(word for word in
cleaned_text.split(' ') if word not in stop_words)
    stemmed_text = ' '.join(apply_stemming(word) for word
in removed_stopwords_text.split(' '))
    return stemmed_text


vectorizer = joblib.load('data/vectorizer.joblib')



In [15]:
processed_txt = preprocess_and_clean(txt)
x = pd.DataFrame([processed_txt], columns = ['preprocessed_text'])
X_train = vectorizer.transform(x['preprocessed_text'])


y_pred_proba = model.predict_proba(X_train)
top3_indices = np.argsort(y_pred_proba[0])[::-1][:3]
top3_categories = model.classes_[top3_indices]
top3_probabilities = y_pred_proba[0][top3_indices]


for category, probability in zip(top3_categories, top3_probabilities):
    print(f"Category: {category}, Probability: {probability}")

Category: politics, Probability: 0.9543413842138971
Category: crime, law and justice, Probability: 0.0273375297423233
Category: society, Probability: 0.01259416880481507
